## Merge MS/MS signals from different polarizations (Pos and Neg) and column types (RP and HILIC) together and add columns to indicate data source ##

Text here

In [26]:
import openpyxl
import os

In [27]:
outputfile = 'satuaivot_peaks.xlsx'

In [28]:
datadir = 'D:\JYU_2019_raw data\\'

In [29]:
PATHS = {}
for f in os.scandir(datadir):
    if ("rp" in f.name.lower()):
        if ("pos" in f.name.lower()):
            arr = [a for a in os.scandir(os.path.join(datadir, f))]
            arr.sort(key = lambda k: os.path.getmtime(k), reverse = True)
            for ff in arr:
                if ("area" in ff.name.lower()):
                    PATHS["RP_Pos"] = ff.path
        elif ("neg" in f.name.lower()):
            arr = [a for a in os.scandir(os.path.join(datadir, f))]
            arr.sort(key = lambda k: os.path.getmtime(k), reverse = True)
            for ff in arr:
                if ("area" in ff.name.lower()):
                    PATHS["RP_Neg"] = ff.path
                    
    elif ("hilic" in f.name.lower()):
        if ("pos" in f.name.lower()):
            arr = [a for a in os.scandir(os.path.join(datadir, f))]
            arr.sort(key = lambda k: os.path.getmtime(k), reverse = True)
            for ff in arr:
                if ("area" in ff.name.lower()):
                    PATHS["HILIC_Pos"] = ff.path
        elif ("neg" in f.name.lower()):
            arr = [a for a in os.scandir(os.path.join(datadir, f))]
            arr.sort(key = lambda k: os.path.getmtime(k), reverse = True)
            for ff in arr:
                if ("area" in ff.name.lower()):
                    PATHS["HILIC_Neg"] = ff.path

In [30]:
ARRAYS = {}
METADATA = {}
for k, p in PATHS.items():
    f = open(p, 'r')
    if (f.readable()):
        column = k.split("_")[0]
        iontype = k.split("_")[1]
        arr = []
        met = []
        lines = f.readlines()
        i = True
        for l in lines:
            if (l[0] == "\t"):
                l = "\t\t" + l
                met.append(l.split('\t'))
            else:
                if (i):
                    l = "Column\tIon type\t" + l
                    i = False
                    met.append(l.split('\t'))
                else:
                    l = f"{column}\t{iontype}\t" + l
                    arr.append(l.split('\t'))
        METADATA[k] = met
        ARRAYS[k] = arr
    f.close()

In [31]:
MASTERARRAY = []
MASTERARRAY += METADATA["RP_Pos"]
for k, v in ARRAYS.items():
    MASTERARRAY += v

In [32]:
MA_new = []
i = MASTERARRAY[0].index("NA")
for l in MASTERARRAY:
    MA_new.append(l[:i])
MASTERARRAY = MA_new

**OPTIONAL**
Remove blank samples from data altogether

In [33]:
indexes = []
MA_new = []
for i, c in enumerate(MASTERARRAY[0]):
    if ("blank" in "" + c.lower()):
        indexes.append(i)
for l in MASTERARRAY:
    newl = [c for i, c in enumerate(l) if i not in indexes]
    MA_new.append(newl)
MASTERARRAY = MA_new

Write merged features to a new .xlsx-file

In [35]:
wb = openpyxl.Workbook()
ws = wb.active

for i, r in enumerate(MASTERARRAY):
    for ii, c in enumerate(r):
        ws.cell(row = i+1, column = ii+1, value = c)

wb.save(os.path.join(datadir, outputfile))
        
##t = open(os.path.join(datadir, outputfile), 'w')
##for l in MASTERARRAY:
##    t.write('\t'.join(l) + '\n')
##t.close()

That should be it!